# Sprint 7: Predicting Mobile Plan Selection for Megaline


## Project Overview
Megaline, a mobile carrier, has discovered that many subscribers are still using legacy plans. To increase adoption of their newer offerings — **Smart** and **Ultra** — the company aims to develop a machine learning model that can accurately recommend the most suitable plan for each user based on their historical behavior.

This project involves building and evaluating classification models to predict whether a subscriber should be on the **Ultra** plan (`is_ultra = 1`) or the **Smart** plan (`is_ultra = 0`). The dataset contains monthly usage statistics for subscribers who have already switched to one of the new plans.  

## Objective
- **Goal:** Develop a model with the highest possible accuracy, with a minimum threshold of **0.75** accuracy on the test set.
- **Approach:**
  1. Explore and understand the dataset.
  2. Split the data into training, validation, and test sets.
  3. Experiment with various classification models and hyperparameters.
  4. Evaluate models using accuracy as the primary metric.
  5. Select the best-performing model and validate it on the test set.
  6. Perform a sanity check to ensure model reliability.

## Dataset Description
The dataset (`users_behavior.csv`) contains the following features:
- **calls** — Number of calls per month  
- **minutes** — Total monthly call duration (in minutes)  
- **messages** — Number of text messages per month  
- **mb_used** — Internet traffic used in MB per month  
- **is_ultra** — Target variable: 1 for Ultra plan, 0 for Smart plan  

## Evaluation Criteria
The final solution will be assessed based on:
- Proper data inspection and preprocessing.
- Correct data splitting and set size justification.
- Comprehensive model experimentation with hyperparameter tuning.
- Clear reporting of findings and accuracy results.
- Code clarity, structure, and readability.

By completing this project, we will not only produce a high-performing classification model but also demonstrate a structured approach to solving a real-world business problem using machine learning.


# 2.1 Explore and understand the dataset.

In [1]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 2: Load the dataset
df = pd.read_csv('/datasets/users_behavior.csv')

# Step 3: Inspect the data
display(df.head())
display(df.info())
display(df.describe())


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


# 2.2 Split the data into training, validation, and test sets.

In [2]:
# Define features and target
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

# Split into train, validation, and test sets
# First split train+valid and test
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345
)

# Then split train and validation
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345
)

# Check shapes
print("Train set size:", features_train.shape)
print("Validation set size:", features_valid.shape)
print("Test set size:", features_test.shape)

Train set size: (1928, 4)
Validation set size: (643, 4)
Test set size: (643, 4)


# 2.3 Experiment with various classification models and hyperparameters. 

In [3]:
# Decision Tree Model
best_tree_score = 0
best_tree_depth = None
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    score = accuracy_score(target_valid, predictions_tree)
    if score > best_tree_score:
        best_tree_score = score
        best_tree_depth = depth
        
print(f"Best Decision Tree: depth={best_tree_depth}, accuracy={best_tree_score:.4f}")

Best Decision Tree: depth=7, accuracy=0.7745


In [4]:
# Random Forest Model
best_forest_score = 0
best_forest_params = (None, None)
for est in range(10, 101, 10):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions_forest = model_forest.predict(features_valid)
        score = accuracy_score(target_valid, predictions_forest)
        if score > best_forest_score:
            best_forest_score = score
            best_forest_params = (est, depth)
            
print(f"Best Random Forest: n_estimators:{best_forest_params[0]}, depth={best_forest_params[1]}, accuracy={best_forest_score:.4f}")

Best Random Forest: n_estimators:50, depth=10, accuracy=0.7978


In [5]:
# Logistic Regression Model
model_logreg = LogisticRegression(random_state=12345, solver='liblinear')
model_logreg.fit(features_train, target_train)
predictions_logreg = model_logreg.predict(features_valid)
logreg_score = accuracy_score(target_valid, predictions_logreg)

print(f"Logistic Regression accuracy={logreg_score:.4f}")

Logistic Regression accuracy=0.7294


# 2.4 Evaluate models using accuracy as the primary metric.

In [6]:
print(f"Best Decision Tree: depth={best_tree_depth}, accuracy={best_tree_score:.4f}")
print()
print(f"Best Random Forest: n_estimators:{best_forest_params[0]}, depth={best_forest_params[1]}, accuracy={best_forest_score:.4f}")
print()
print(f"Logistic Regression accuracy={logreg_score:.4f}")

Best Decision Tree: depth=7, accuracy=0.7745

Best Random Forest: n_estimators:50, depth=10, accuracy=0.7978

Logistic Regression accuracy=0.7294


# 2.5 Select the best-performing model and validate it on the test set.

In [7]:
# Train the model on train+validation and test it

# Best model from testing
best_model = RandomForestClassifier(
    random_state=12345,
    n_estimators=best_forest_params[0],
    max_depth=best_forest_params[1]
)

best_model.fit(features_train_valid, target_train_valid)
test_predictions = best_model.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)

print(f"Test set accuracy: {test_accuracy:.4f}")

Test set accuracy: 0.7947


# 2.6 Perform a sanity check to ensure model reliability.

In [8]:
# Sanity check with random predictions
random_preds = np.random.randint(0, 2, size=len(target_test))
random_accuracy = accuracy_score(target_test, random_preds)

print(f"Random baseline accuracy: {random_accuracy:.4f}")
print(f"Our model improvement: {test_accuracy - random_accuracy:.4f}")

Random baseline accuracy: 0.5054
Our model improvement: 0.2893


# Conclusion & Recommendations

### Key Findings
- **Objective Achieved:**  
  The machine learning model surpassed the required **0.75 accuracy threshold** on the test dataset, delivering a performance level that is reliable for production deployment.

- **Best Model Identified:**  
  The **Random Forest Classifier** emerged as the top performer after testing multiple algorithms and tuning hyperparameters.
  - **Best Parameters:** `n_estimators = 50`, `max_depth = 10}`
  - **Validation Accuracy:** `0.7947`
  - **Test Accuracy:** `0.7978`

- **Performance Benchmark:**  
  The selected model significantly outperformed the random baseline accuracy of 0.5163, confirming that the predictions are meaningful and not due to chance.

### Business Impact
- This solution enables **personalized plan recommendations** for Megaline’s subscribers based on real usage data.
- By automating the plan recommendation process:
  - **Customer Satisfaction** is expected to increase through better plan fit.
  - **Revenue Growth** is achievable by upselling subscribers who would benefit from the Ultra plan.
  - **Operational Efficiency** improves by reducing the need for manual plan assessments.

### Recommendations for Next Steps
1. **Integration:** Deploy the Random Forest model into Megaline’s CRM system to provide real-time plan recommendations.
2. **Monitoring:** Implement performance tracking to ensure model accuracy remains above the 0.75 threshold over time.
3. **Enhancement:**  
   - Gather more behavioral features (e.g., roaming data, time-of-day usage) to improve accuracy.
   - Explore additional algorithms like Gradient Boosting for potential gains.
4. **Customer Feedback Loop:** Use subscriber feedback to refine recommendations and further personalize plan offerings.

---
**Final Statement:**  
The project successfully delivers a high-performing classification model that is both robust and practical for business use, positioning Megaline to optimize its customer plan allocation strategy with data-driven insights.
